<a href="https://colab.research.google.com/github/nva1dman/ST8_to_FITS/blob/main/spectra_code_st8_to_fits.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Обновление от 2024-09-02**

Код, предназначенный для перевода данных из формата ST-8 в формат FITS, с добавлением информании в header объекта

In [ ]:
# Чтение файла и обработка заголовка
file_st8_path = "335r021008-15.ST8"

with open(file_st8_path, 'rb') as f:
    # Чтение первых 2048 байт (заголовок)
    header = f.read(2048)

    # Извлечение метаданных из заголовка
    header_text = header.decode('ascii', errors='ignore')

    # Чтение оставшейся части файла как данные изображения
    image_data_compressed = f.read()

# Отображение первых 20 строк заголовка для проверки параметров
for line in header_text.splitlines()[:20]:
    print(line)

# Логика декомпрессии изображения

В этом коде выполняется декомпрессия изображения, сжатого с использованием специфического алгоритма. Основная цель — восстановить двумерный массив пикселей изображения. Вот пошаговая логика:

1. **Инициализация массива изображения**:
   Создается пустой двумерный массив размером *height* x *width*, который будет содержать декомпрессированные данные пикселей.
    
2. **Чтение длины сжатой строки**:
   Для каждой строки изображения сначала считываются два байта, которые содержат длину сжатой строки. Эти байты интерпретируются как длина, и указатель данных сдвигается.
    
3. **Декомпрессия строки**:
   Каждый пиксель строки обрабатывается по очереди:
   - *Первый пиксель строки* всегда записывается напрямую из сжатых данных.
   - Для *остальных пикселей* возможны два варианта:
     - Если байт данных равен 128 (или 0x80), это указывает на то, что следующий пиксель нужно записать напрямую.
     - В противном случае, байт данных интерпретируется как "дельта", которая добавляется к предыдущему значению пикселя для восстановления текущего.
    
4. **Обработка дельты**:
   Если байт данных содержит дельту (разницу), она может быть как положительной, так и отрицательной. Если значение дельты больше 127, оно интерпретируется как отрицательное число, используя дополнительный код (two's complement).

5. **Заполнение массива**:
   Для каждого ряда строк декомпрессированные пиксели записываются в массив. Этот процесс повторяется для всех строк изображения.
    
6. **Проверка результата**:
   После декомпрессии данных программа выводит форму итогового изображения и небольшой фрагмент массива, чтобы убедиться в правильности декомпрессии.

Таким образом, этот алгоритм восстанавливает исходное изображение из сжатых данных, используя дельты между соседними пикселями для уменьшения объема хранимых данных.

In [ ]:
import numpy as np

def decompress_sbig_image(data, width, height):
    # Инициализация пустого массива для изображения
    image = np.zeros((height, width), dtype=np.uint16)

    # Указатель на текущую позицию в сжатых данных
    data_index = 0

    for row in range(height):
        # Чтение длины сжатой строки
        compressed_length = data[data_index] + (data[data_index + 1] << 8)
        data_index += 2

        pixel_index = 0
        while pixel_index < width:
            if data_index >= len(data):
                break

            # Первый пиксель записывается напрямую
            if pixel_index == 0:
                image[row, pixel_index] = data[data_index] + (data[data_index + 1] << 8)
                data_index += 2
            else:
                delta = data[data_index]
                if delta == 128:  # 0x80 указывает, что следующее значение записано напрямую
                    image[row, pixel_index] = data[data_index + 1] + (data[data_index + 2] << 8)
                    data_index += 3
                else:
                    # delta записан в виде числа в дополнительном коде (2's complement)
                    if delta > 127:
                        delta -= 256
                    image[row, pixel_index] = image[row, pixel_index - 1] + delta
                    data_index += 1

            pixel_index += 1

    return image

# Извлечение ширины и высоты из заголовка
width = 1530  # Взято из предыдущего анализа
height = 1020

# Декомпрессия данных изображения
image_data = decompress_sbig_image(image_data_compressed, width, height)

# Проверка формы массива и вывод небольшой части декомпрессированного изображения
image_data.shape, image_data[:5, :5]  # Отображение формы и небольшой части изображения

In [ ]:
from astropy.io import fits
from astroquery.simbad import Simbad
from astropy.time import Time
import astropy.units as u
from astropy.coordinates import SkyCoord

# Чтение данных изображения и создание объекта PrimaryHDU
hdu = fits.PrimaryHDU(data=image_data)
header = hdu.header

# Заполнение заголовка параметрами из исходного файла
header['SIMPLE'] = True
header['BITPIX'] = 16
header['NAXIS'] = 2
header['NAXIS1'] = width
header['NAXIS2'] = height
header['TELESCOP'] = 'SBIG ST-8'
header['FILE_VER'] = '3'
header['DATA_VER'] = '1'
header['EXPOSURE'] = 90000
header['FOC_LEN'] = 80.000
header['APERTURE'] = 35.0000
header['RESP_FAC'] = 3000.000
header['NOTE'] = '-'
header['BACKGRND'] = 42
header['RANGE'] = 545
header['DATE-OBS'] = '2002-10-09'
header['TIME'] = '01:19:47'
header['EXP_STATE'] = 293
header['CCD_TEMP'] = -16.51
header['NUM_EXPS'] = 1
header['EACH_EXP'] = 90000
header['XPIXSZ'] = 0.0090
header['YPIXSZ'] = 0.0090
header['PEDESTAL'] = 0
header['E_GAIN'] = 2.42

# Запрашиваем название объекта у пользователя
object_name = input("Введите название объекта для поиска в Simbad: ")

try:
    # Поиск объекта в базе данных Simbad
    result_table = Simbad.query_object(object_name)

    if result_table is not None:
        ra = result_table['RA'][0]  # Получаем прямое восхождение
        dec = result_table['DEC'][0]  # Получаем склонение

        # Преобразуем RA и DEC в градусы
        coord = SkyCoord(ra, dec, unit=(u.hourangle, u.deg), frame='icrs')

        # Обновляем заголовок FITS файла
        header['RA'] = coord.ra.deg
        header['DEC'] = coord.dec.deg
        header['EPOCH'] = 2000
        header['DISPAXIS'] = 1
        header['OBSERVAT'] = 'Fesenkov Astrophysical Institute'
        header['OBJECT'] = object_name
        header['VOBS'] = 0
        header['OBSERVER'] = 'Valiullin Rashit'

        # Вычисление UT и других параметров
        ut = Time(header['DATE-OBS'] + ' ' + header['TIME']).ut1
        header['UT'] = ut.iso

        # Генерация имени файла в формате "ObjectName_08-09.06.1969_20m.fit"
        date_obs = header['DATE-OBS'].replace('-', '.')
        exposure_minutes = int(header['EXPOSURE'] / 60)  # Преобразуем экспозицию в минуты
        fits_file_path = f"{object_name.replace(' ', '_')}_{date_obs}_{exposure_minutes}m.fits"
        hdu.writeto(fits_file_path, overwrite=True)

        print(f"Файл сохранен как {fits_file_path}")
    else:
        print(f"Объект '{object_name}' не найден в базе данных Simbad.")
        print("Попробуйте другое написание или используйте альтернативные имена объекта.")
except Exception as e:
    print(f"Произошла ошибка: {e}")